<a href="https://colab.research.google.com/github/amitthakur4/ann/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

# PART1 - DATA PREPROCESSING
IMPORTING THE DATASET


In [ ]:
dataset= pd.read_csv(r'/content/sample_data/Churn_Modelling.csv')
X= dataset.iloc[:,3:-1].values
y= dataset.iloc[:,-1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


In [ ]:
dataset.isnull().values.any()

False

### Encoding Categorical Data





In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
X[:,2]= le.fit_transform(X[:,2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding to Geographical Column


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [1])], remainder="passthrough")
X= np.array(ct.fit_transform(X))

In [ ]:
X[1]

array([0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
      dtype=object)

### Splitting the dataset into traning nd test dataset


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size = 0.2, random_state=0)

In [ ]:
print(f"size of train is {X_train.shape[0]} & size of test is {y_test.shape[0]}")

size of train is 8000 & size of test is 2000


In [ ]:
## NOTE: Feature Scaling is very necessary for ANN
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

# Part 2: Building the ANN

## Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()
### this will allow our ann to initiate sequentially 

## Adding the input layer and the 1st hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6 , activation= 'relu'))
# we are adding 1st hidden layer with 6 neuron and activation function of rectifier - add method can add layer at whatevr stage of your nn
# question . y 6 ? there is no thumb rule its has to be hyper tuned 

## Adding 2nd hidden layer 

In [ ]:
ann.add(tf.keras.layers.Dense(units=6 , activation= 'relu'))

## Adding Output Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1 , activation= 'sigmoid'))
# dependent var is Exited : which is binary hence sigmoid and only one dependent var hence unit= 1

# Part 3: Training the ANN

## Compiling the ANN

In [ ]:
# lets start compiling with optimizer , loss fun and matric ( i.e Accuarcy bcoz of binary problem)
ann.compile(optimizer= 'adam' , loss= 'binary_crossentropy' , metrics= ['accuracy'])
""" 
adam - gonna perform stochastic gradient descent
loss - binary_crossentropy for binary , for multiple dimention binary - categorical_crossentropy and activation above should be - softmax
metrics - we can pass a list 
"""

' \nadam - gonna perform stochastic gradient descent\nloss - binary_crossentropy for binary , for multiple dimention binary - categorical_crossentropy and activation above should be - softmax\nmetrics - we can pass a list \n'

## Traning the ANN on Traning dataset

In [ ]:
ann.fit(X_train, y_train, batch_size= 64 , epochs= 100)
# traning always has to be done with fit method
# its always better to do batch size rather than row by row and ideal batch size should be 32
# nn always needs some good epochs to train 

Epoch 1/100
125/125 [==============================] - 0s 1ms/step - loss: 0.5649 - accuracy: 0.7921
Epoch 2/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4838 - accuracy: 0.7960
Epoch 3/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4573 - accuracy: 0.7960
Epoch 4/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4453 - accuracy: 0.7960
Epoch 5/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4392 - accuracy: 0.7960
Epoch 6/100
125/125 [==============================] - 0s 2ms/step - loss: 0.4353 - accuracy: 0.7961
Epoch 7/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4326 - accuracy: 0.7991
Epoch 8/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4301 - accuracy: 0.8029
Epoch 9/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4279 - accuracy: 0.8067
Epoch 10/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4255 - accura

In [ ]:
#predicting for single observation 
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000, 2,1,1,50000]]))

array([[0.03863409]], dtype=float32)

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000, 2,1,1,50000]]))>0.5)

[[False]]


## Predicting the test set result

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1516   79]
 [ 207  198]]


0.857